# Multiclass Models Runner

This notebook needs: 
* raw CSV at DATA_PATH with target column named phase. 

This notebook automatically:
* encodes categoricals (one-hot),
* preserves a mapping of class labels,
* calls each model script,
* saves summaries.

## Dataset

### Imports & paths

In [1]:
import os
import numpy as np
import pandas as pd

import inspect
from joblib import dump
import os

from numpy.random import RandomState
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.utils.multiclass import unique_labels

from utils_ml import ensure_dirs

# Model runners
from rf_model_base import run_random_forest
from xgb_model_base import run_xgboost
from catboost_model_base import run_catboost
from logreg_model_base import run_logreg
from svm_model_base import run_svm_rbf

DATA_PATH = "data/dataset_deduplicated_original.csv"   
MODEL_DIR = "./data/original_base/org_baase_best_ML_pkl"
PERF_DIR  = "./data/original_base/org_base_ML_performance_summary"

ensure_dirs(MODEL_DIR, PERF_DIR)
print("Setup complete.")


d:\OnLine Courses\LLMs for Materials and Chemistry Hackaton\LLMHackathon\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setup complete.


### Load dataset

In [2]:
df = pd.read_csv(DATA_PATH)
assert "phase" in df.columns, "Target column 'phase' not found."

y_raw = df["phase"].astype(str)
X_raw = df.drop(columns=["phase"])

# Identify column types
cat_cols = X_raw.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
num_cols = X_raw.select_dtypes(include=[np.number]).columns.tolist()
print(f"Detected {len(num_cols)} numeric and {len(cat_cols)} categorical features.")


Detected 230 numeric and 0 categorical features.


### Train/Valid/Test split (stratified)

In [3]:
def stratified_train_valid_test_split(
    X, y, *, test_size=0.2, valid_size=0.2, min_per_class=3, random_state=42
):
    """
    Manual stratified 3-way split that avoids sklearn's '>=2 per class per split' constraint.
    - Pre-merges ultra-rare classes (< min_per_class) into 'Other'.
    - Ensures at least 1 sample per class in each split whenever possible.
    Returns: X_tr, X_va, X_te, y_tr, y_va, y_te, merged_flag
    Works with X as DataFrame/ndarray; y as array-like/Series.
    """
    # --- normalize inputs ---
    is_df = isinstance(X, pd.DataFrame)
    y = pd.Series(y).astype(str)
    rs = RandomState(random_state)

    # --- pre-merge rare classes ---
    vc = y.value_counts()
    rare = vc[vc < min_per_class].index.tolist()
    merged_flag = False
    if rare:
        print(f"Merging rare classes into 'Other': {rare}")
        y = y.where(~y.isin(rare), other="Other")
        merged_flag = True

    # if 'Other' has only 1 sample, drop it (can't be split sensibly)
    if (y == "Other").sum() == 1:
        print("Dropping single-sample 'Other' to enable splitting.")
        keep = ~(y == "Other")
        X = X.loc[keep] if is_df else X[keep]
        y = y.loc[keep]

    n = len(y)
    n_valid = max(1, int(round(valid_size * n)))
    n_test  = max(1, int(round(test_size * n)))
    # keep at least 1 sample for train
    n_train = n - n_valid - n_test
    if n_train < 1:
        # borrow from the larger of valid/test
        take_from_valid = n_valid >= n_test
        if take_from_valid and n_valid > 1:
            n_valid -= 1
        elif n_test > 1:
            n_test -= 1
        n_train = n - n_valid - n_test
        if n_train < 1:
            # final fallback
            n_train, n_valid, n_test = max(1, n-2), 1, 1

    # --- per-class allocation ---
    idx = np.arange(n)
    if is_df:
        idx = X.index.to_numpy()

    train_idx = []
    valid_idx = []
    test_idx  = []

    for cls, cls_count in y.value_counts().items():
        cls_indices = idx[(y == cls).to_numpy()]
        rs.shuffle(cls_indices)

        # proportional targets
        v = int(round(valid_size * cls_count))
        t = int(round(test_size  * cls_count))
        # ensure at least 1 where possible
        v = min(max(1 if cls_count >= 3 else 0, v), cls_count)  # allow 0 if class size < 3
        t = min(max(1 if cls_count >= 3 else 0, t), cls_count - v)

        r = cls_count - v - t
        if cls_count >= 3:
            if r < 1:
                # borrow from the larger of v/t
                if v > t and v > 1:
                    v -= 1
                elif t > 1:
                    t -= 1
                r = cls_count - v - t
                if r < 1:
                    # last resort: set v=t=1
                    v, t, r = 1, 1, cls_count - 2
        else:
            # for tiny classes (size 1 or 2), just push all to train
            v, t, r = 0, 0, cls_count

        # slice
        cls_valid = cls_indices[:v]
        cls_test  = cls_indices[v:v+t]
        cls_train = cls_indices[v+t:]

        valid_idx.append(cls_valid)
        test_idx.append(cls_test)
        train_idx.append(cls_train)

    train_idx = np.concatenate(train_idx) if train_idx else np.array([], dtype=int)
    valid_idx = np.concatenate(valid_idx) if valid_idx else np.array([], dtype=int)
    test_idx  = np.concatenate(test_idx)  if test_idx  else np.array([], dtype=int)

    # If global counts drifted from targets, that's ok; we guaranteed per-class feasibility.

    # --- build splits ---
    def take(Xin, ind):
        return Xin.loc[ind] if is_df else Xin[np.isin(idx, ind)]

    X_tr = take(X, train_idx)
    X_va = take(X, valid_idx)
    X_te = take(X, test_idx)
    y_tr = y.loc[train_idx]
    y_va = y.loc[valid_idx]
    y_te = y.loc[test_idx]

    # sanity prints (optional)
    print("Final per-split counts:", len(y_tr), len(y_va), len(y_te))
    print("Min per-class (train/valid/test):",
          y_tr.value_counts().min() if len(y_tr) else 0,
          y_va.value_counts().min() if len(y_va) else 0,
          y_te.value_counts().min() if len(y_te) else 0)

    return X_tr, X_va, X_te, y_tr, y_va, y_te, merged_flag


In [4]:
X_train_raw, X_valid_raw, X_test_raw, y_train, y_valid, y_test, merged = stratified_train_valid_test_split(
    X_raw, y_raw, test_size=0.2, valid_size=0.2, min_per_class=3, random_state=42
)

print("Merged rare classes into 'Other'?", merged)
print("Class counts (train/valid/test):")
print(y_train.value_counts(), "\n", y_valid.value_counts(), "\n", y_test.value_counts())
print("Shapes:", X_train_raw.shape, X_valid_raw.shape, X_test_raw.shape)


Merging rare classes into 'Other': ['layered rock salt', 'double perovskite', 'c-type', 'monosilicates', 'tetragonal', 'wurtzite', 'rutile-type', 'hexagonal', 'scheelite', 'p2-type layered', 'disilicates', 'columbite', 'pseudocubic t phase structure']
Final per-split counts: 447 150 150
Min per-class (train/valid/test): 1 1 1
Merged rare classes into 'Other'? True
Class counts (train/valid/test):
phase
spinel                         137
cubic perovskite                73
fluorite                        62
multiphase                      44
pyrochlore                      24
rock salt                       19
orthorhombic perovskite         13
magnetoplumbite                 11
amorphous                       10
Other                           10
monoclinic                      10
bixbyite                         8
tetragonal perovskite            8
layered ruddlesdenâpopper      5
garnet                           5
rutile                           4
o3-type layered                  3

### Preprocessing: Impute + One-Hot for categoricals, Impute for numerics

One-hot encodes all categorical features once (fit on train), passes through numerics. Linear/SVM add scaling internally; trees use the one-hot features as-is.

In [5]:
# --- OneHotEncoder: version-safe 'sparse_output' vs 'sparse' ---
if "sparse_output" in inspect.signature(OneHotEncoder).parameters:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True)
else:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=True)

# Pipelines per type (fit on TRAIN only to avoid leakage)
cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", ohe),
])

num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
])

preprocess = ColumnTransformer(
    transformers=[
        ("cat", cat_pipe, cat_cols),
        ("num", num_pipe, num_cols),
    ],
    remainder="drop",
    sparse_threshold=1.0,   # keep output sparse when beneficial
)

# Fit on training data only
preprocess.fit(X_train_raw)

def transform_and_get_feature_names(X_df):
    Xtx = preprocess.transform(X_df)
    # Build feature names
    if cat_cols:
        cat_names = preprocess.named_transformers_["cat"].named_steps["ohe"].get_feature_names_out(cat_cols)
    else:
        cat_names = np.array([])
    feat_names = np.concatenate([cat_names, np.array(num_cols)])
    return Xtx, feat_names

X_train, feature_names = transform_and_get_feature_names(X_train_raw)
X_valid, _ = transform_and_get_feature_names(X_valid_raw)
X_test,  _ = transform_and_get_feature_names(X_test_raw)

# Class labels (string) — unchanged
class_labels = unique_labels(y_train, y_valid, y_test)
print(f"Classes: {list(class_labels)}")
print("Preprocessing complete. Shapes ->",
      "train:", X_train.shape, "valid:", X_valid.shape, "test:", X_test.shape)


Classes: [np.str_('Other'), np.str_('amorphous'), np.str_('bixbyite'), np.str_('cubic'), np.str_('cubic perovskite'), np.str_('fluorite'), np.str_('garnet'), np.str_('layered ruddlesdenâ\x80\x93popper'), np.str_('magnetoplumbite'), np.str_('monoclinic'), np.str_('multiphase'), np.str_('o3-type layered'), np.str_('orthorhombic perovskite'), np.str_('pyrochlore'), np.str_('rock salt'), np.str_('rutile'), np.str_('spinel'), np.str_('tetragonal perovskite')]
Preprocessing complete. Shapes -> train: (447, 230) valid: (150, 230) test: (150, 230)


### Feature normalization

In [6]:
SCALER_PATH = os.path.join(MODEL_DIR, "org_global_std_scaler.pkl")

scaler = StandardScaler()          # scales each feature by its max |value| on TRAIN
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test  = scaler.transform(X_test)

# (Optional) save for inference pipelines
dump(scaler, SCALER_PATH)
print("Applied Standard scaling to X_train/valid/test. Saved scaler ->", SCALER_PATH)


Applied Standard scaling to X_train/valid/test. Saved scaler -> ./data/original_base/org_baase_best_ML_pkl\org_global_std_scaler.pkl


### Encode target feature

In [7]:
le = LabelEncoder().fit(pd.concat([y_train, y_valid, y_test]).astype(str))
y_train_enc = le.transform(y_train.astype(str))
y_valid_enc = le.transform(y_valid.astype(str))
y_test_enc  = le.transform(y_test.astype(str))

# numeric class labels 0..K-1 for metrics & models
class_labels = np.arange(len(le.classes_))

# (optional) save the mapping for later interpretability
mapping = pd.DataFrame({"class_index": class_labels, "class_name": le.classes_})
mapping.to_csv("data/original_base/org_base_ML_performance_summary/label_mapping.csv", index=False)
mapping.head()


,class_index,class_name
0,0,Other
1,1,amorphous
2,2,bixbyite
3,3,cubic
4,4,cubic perovskite


## Run all models

**CV & grids:** 3-fold CV with compact grids keeps runtime reasonable but still tunes the big levers.

**Refit policy:** Each script tunes on the train split (via CV), evaluates the tuned model on the validation split, then refits the best params on train+validation and reports test metrics in the final summary. The saved .pkl is this refit model (best performance for deployment).

In [8]:
all_metrics = []
all_top10 = []

# Tree-based ensembles
m, t = run_random_forest(X_train, y_train_enc, X_valid, y_valid_enc, X_test, y_test_enc,
                         feature_names, class_labels, MODEL_DIR)
all_metrics.append(m); all_top10.append(t)

m, t = run_xgboost(X_train, y_train_enc, X_valid, y_valid_enc, X_test, y_test_enc,
                   feature_names, class_labels, MODEL_DIR)
all_metrics.append(m); all_top10.append(t)

m, t = run_catboost(X_train, y_train_enc, X_valid, y_valid_enc, X_test, y_test_enc,
                    feature_names, class_labels, MODEL_DIR)
all_metrics.append(m); all_top10.append(t)

# Regularized linear
m, t = run_logreg(X_train, y_train_enc, X_valid, y_valid_enc, X_test, y_test_enc,
                  feature_names, class_labels, MODEL_DIR)
all_metrics.append(m); all_top10.append(t)

# SVM RBF
m, t = run_svm_rbf(X_train, y_train_enc, X_valid, y_valid_enc, X_test, y_test_enc,
                   feature_names, class_labels, MODEL_DIR)
all_metrics.append(m); all_top10.append(t)

print("All models trained and evaluated.")


d:\OnLine Courses\LLMs for Materials and Chemistry Hackaton\LLMHackathon\Lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
d:\OnLine Courses\LLMs for Materials and Chemistry Hackaton\LLMHackathon\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\OnLine Courses\LLMs for Materials and Chemistry Hackaton\LLMHackathon\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\OnLine Courses\LLMs for Materials and Chemistry Hackaton\LLMHackathon\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not con

All models trained and evaluated.


## Model Performance

### Performance summary table (test-set metrics)

In [9]:
# Flatten confusion matrices to strings for compact CSV; keep scores as columns
rows = []
for m in all_metrics:
    row = {
        "model": m.get("model"),
        "precision_weighted": m.get("precision_weighted"),
        "recall_weighted": m.get("recall_weighted"),
        "f1_weighted": m.get("f1_weighted"),
        "precision_macro": m.get("precision_macro"),
        "recall_macro": m.get("recall_macro"),
        "f1_macro": m.get("f1_macro"),
        "auc_macro": m.get("auc_macro"),
        "auc_weighted": m.get("auc_weighted"),
        "kappa": m.get("kappa"),
        "best_params": m.get("best_params"),
    }
    cm = m.get("confusion_matrix")
    if cm is not None:
        row["confusion_matrix"] = ";".join([",".join(map(str, r)) for r in cm])
    else:
        row["confusion_matrix"] = ""
    rows.append(row)

perf_df = pd.DataFrame(rows)
perf_path = os.path.join(PERF_DIR, "performance_summary.csv")
perf_df.to_csv(perf_path, index=False)
perf_df


,model,precision_weighted,recall_weighted,f1_weighted,precision_macro,recall_macro,f1_macro,auc_macro,auc_weighted,kappa,best_params,confusion_matrix
0,RandomForest,0.673749,0.733333,0.694726,0.569553,0.596878,0.566708,0.918602,0.930988,0.679932,"{'n_estimators': 200, 'max_depth': None, 'min_...","0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0;0,0,0,0,0,..."
1,XGBoost,0.713072,0.753333,0.719576,0.592124,0.573792,0.566880,0.901936,0.932215,0.704126,"{'n_estimators': 300, 'learning_rate': 0.1, 'm...","0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,0;0,3,0,0,0,..."
2,CatBoost,0.655607,0.720000,0.665562,0.514134,0.511543,0.495926,0.905310,0.924121,0.660451,"{'iterations': 300, 'learning_rate': 0.15, 'de...","0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,0;0,0,0,0,0,..."
3,LogisticRegression,0.721658,0.773333,0.737125,0.616553,0.645511,0.614969,0.886788,0.923074,0.729644,"{'penalty': 'l2', 'C': 1.0}","0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,0;0,2,0,0,0,..."
4,SVM_RBF,0.716041,0.740000,0.675982,0.554871,0.543739,0.510362,0.944313,0.935454,0.683681,"{'C': 1.0, 'gamma': 'scale'}","0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,0;0,0,0,0,0,..."
